In [29]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

import  nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import random

In [13]:
data = "long ago , the mice had a general council to consider what measures they could take to outwit their common enemy , the cat . some said this , and some said that but at last a young mouse got up and said he had a proposal to make , which he thought would meet the case . you will all agree , said he , that our chief danger consists in the sly and treacherous manner in which the enemy approaches us . now , if we could receive some signal of her approach , we could easily escape from her . i venture , therefore , to propose that a small bell be procured , and attached by a ribbon round the neck of the cat . by this means we should always know when she was about , and could easily retire while she was in the neighbourhood . this proposal met with general applause , until an old mouse got up and said that is all very well , but who is to bell the cat ? the mice looked at one another and nobody spoke . then the old mouse said it is easy to propose impossible remedies."

# Conveting dataset into valuable features to be used in the model
def feature_extraction(dataset):
    tokenized_data = word_tokenize(dataset)
    training_data = np.array(tokenized_data)
    count_data = Counter(tokenized_data).most_common()
    dictionary={}
    for i in range(len(count_data)):
        data = count_data[i]
        dictionary[data]=len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))
    return dictionary, reverse_dictionary , training_data

dictionary , reverse_dictionary , training_data = feature_extraction(data)


In [18]:
# Model Parameters
vocab_size = len(dictionary)
n_input = 3
n_hidden = 512
total_epoch=10000
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])
output = {'weights':tf.Variable(tf.random_normal([n_hidden,vocab_size])),'biases':tf.Variable(tf.random_normal([vocab_size]))}


In [35]:
def recurrent_neural_network(x):
    
    x = tf.reshape(x,[-1,n_input])
    x = tf.split(x,n_input,1)
    
    rnn_cell = rnn.BasicLSTMCell(n_hidden,reuse=True)
    # rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    
    return tf.add(tf.matmul(outputs[-1],output['weights']),output['biases'])

In [44]:
def feed_network(x,y):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(cost)
    correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    with tf.Session() as sess:        
        sess.run(tf.global_variables_initializer())
        total_loss = 0
        accuracy = 0
        offset = random.randint(0,len(training_data-4))
        for epoch in range(total_epoch):
            epoch_x = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
            epoch_x = np.reshape(np.array(epoch_x),[-1,n_input,1])
            epoch_y = np.zeros([vocab_size], dtype=float)
            epoch_y[dictionary[str(training_data[offset+n_input])]] = 1.0
            epoch_y = np.reshape(epoch_y,[1,-1])
            
            _,acc,loss = sess.ru([optimizer,accuracy,loss],feed_dict={x:epoch_x,y:epoch_y})
            total_loss += loss
            accuracy += acc
            print('After epoch ',epoch,' total loss ',total_loss)


In [45]:
feed_network(x,y)

ValueError: Variable rnn/basic_lstm_cell/kernel/RMSProp/ already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-27-5e848fe56b64>", line 4, in feed_network
    optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(cost)
  File "<ipython-input-36-0bc652b78f44>", line 1, in <module>
    feed_network(x,y)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
